In [3]:
import pandas as pd
import string
from datamuse import datamuse
from pycorenlp import StanfordCoreNLP 
import json
import nltk
from collections import defaultdict
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


In [ ]:
"""
Extracts the features required for the CWI basline system as described in CWI3G3G2 in (Yimam 2017), with only the most basic frequency features.
1) Number of vowels
2)Number of syllables
3) Number of characters

4) Frewuency in simple Wiki
5)Frquency of word in HIT paragraph
6) Frequency of word in Ngram corpus



"""

In [ ]:
#New preprocessing to handle hyphenated and MWEs
# Now hyphens are treated a seperate words and added to the split column.

location = "cwishareddataset/traindevset/english/Wikipedia_Toy.tsv"
import pandas as pd
import string


data_frame = pd.read_table(location, names=('ID', 'sentence', 'start_index', 'end_index', 'word', 'total_native',
                                            'total_non_native', 'native_complex', 'non_native_complex', 'complex_binary',
                                            'complex_probabilistic'), encoding='utf-8-sig')
data_frame = data_frame.astype(str)

# Cleaning function for words

data_frame['sentence'] = data_frame['sentence'].apply(lambda x: x.replace("%", "percent"))
data_frame['sentence'] = data_frame['sentence'].apply(lambda x: x.replace("’", "'"))


remove = string.punctuation
remove = remove.replace("'", "") + '“”()'
remove = remove.replace("'", "")# don't remove apostrophies 
remove = remove + '“'
remove = remove +'”'
pattern = r"[{}]".format(remove) # create the pattern
word_set['word'] = word_set['word'].apply(lambda x :x.translate({ord(char): None for char in remove}))


# Split the words and add them to the 'split' column, treating hyphens as separate words
data_frame['split'] = data_frame['word'].apply(lambda x: [word for word in x.replace('-', ' - ').split()])

print("Cleaned DataFrame:")
data_frame





In [34]:
# 1- New preprocessing to handle hyphenated and MWEs
# Now hyphens are treated a seperate words and added to the split column.

location = "cwishareddataset/traindevset/english/News_Train.tsv"
import pandas as pd
import string


data_frame = pd.read_table(location, names=('ID', 'sentence', 'start_index', 'end_index', 'word', 'total_native',
                                            'total_non_native', 'native_complex', 'non_native_complex', 'complex_binary',
                                            'complex_probabilistic'), encoding='utf-8-sig')
data_frame = data_frame.astype(str)

# Cleaning function for words

data_frame['sentence'] = data_frame['sentence'].apply(lambda x: x.replace("%", "percent"))
data_frame['sentence'] = data_frame['sentence'].apply(lambda x: x.replace("’", "'"))


remove = string.punctuation
remove = remove.replace("-", "")
remove = remove.replace("(", "")
remove = remove.replace(",", "")
remove = remove.replace("'", "")# don't remove apostrophies 
remove = remove + '“'
remove = remove +'”'
pattern = r"[{}]".format(remove) # create the pattern


# Split the words and add them to the 'split' column, treating hyphens as separate words
data_frame['split'] = data_frame['word'].apply(lambda x: [word for word in x.replace('-', ' - ').split()])


# Split the words and add them to the 'split' column, treating hyphens as separate words
data_frame['split'] = data_frame['word'].apply(lambda x: [word for word in x.replace('-', ' - ').split()])
print("Cleaned DataFrame:")


Cleaned DataFrame:


In [ ]:
#2 speeds up api

import requests
from concurrent.futures import ThreadPoolExecutor

def get_ngram_counts(data_frame):
    abs_counts = []
    rel_counts = []

    session = requests.Session()  # Create a session object for reusing connections

    def process_word(word):
        url = 'https://api.ngrams.dev/eng/search'
        params = {
            'query': word,
            'flags': 'cs',
            'limit': 1
        }

        response = session.get(url, params=params, verify=False)  # Reuse the session for subsequent requests
        data = response.json()

        if 'ngrams' in data and len(data['ngrams']) > 0:
            abs_count = data['ngrams'][0]['absTotalMatchCount']
            rel_count = data['ngrams'][0]['relTotalMatchCount']
        else:
            abs_count = None
            rel_count = None

        return abs_count, rel_count

    words = data_frame['word']
    num_workers = min(len(words), 10)  # Adjust the number of workers as per your requirements
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(process_word, words)

    for abs_count, rel_count in results:
        abs_counts.append(abs_count)
        rel_counts.append(rel_count)

    data_frame['absTotalMatchCount'] = abs_counts
    data_frame['relTotalMatchCount'] = rel_counts

    return data_frame


In [ ]:
get_ngram_counts(data_frame)

In [ ]:
data_frame.to_csv('output_N.csv', index=False)

In [35]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

# Function to get total syllables using Datamuse API
@lru_cache(maxsize=None)
def get_total_syllables(word):
    base_url = 'https://api.datamuse.com/words'
    params = {
        'sp': word,
        'max': 1,
        'md': 'psf'
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if len(data) > 0:
        word_data = data[0]
        if 'word' in word_data and 'numSyllables' in word_data:
            syllables = int(word_data['numSyllables'])
            return syllables

    return 0


# Function to process the DataFrame and print results
def process_dataframe(data_frame):
    data_frame['syllables'] = data_frame['word'].apply(lambda x: sum(get_total_syllables(word) for word in x))
    data_frame['vowels'] = data_frame['word'].apply(lambda x: sum(1 for word in x for char in word if char.lower() in "aeiou"))
    data_frame['characters'] = data_frame['word'].apply(lambda x: sum(len(word) for word in x))

    # Print values for each word
    for index, row in data_frame.iterrows():
        print(f"Word: {row['word']}, Syllables: {row['syllables']}, Vowels: {row['vowels']}, Characters: {row['characters']}")

    # Print the updated DataFrame
    print("Updated DataFrame:")
    print(data_frame)




# Call the function to process the DataFrame
process_dataframe(data_frame)


KeyError: 0

In [ ]:
# Call the function to process the DataFrame
process_dataframe(data_frame)


In [ ]:
import nltk
from collections import defaultdict
from itertools import chain

def search_ngrams(file_path, ngrams):
    # Create a dictionary to store ngram occurrences
    ngram_occurrences = defaultdict(int)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Convert to lowercase and split the line into words
            words = nltk.word_tokenize(line.lower())
            # Generate ngrams
            line_ngrams = set(nltk.ngrams(words, len(next(iter(ngrams)))))
            # Search for each ngram occurrence
            for ngram in line_ngrams:
                if ngram in ngrams:
                    ngram_occurrences[ngram] += 1
                
    return ngram_occurrences

# Create a list of ngrams from the words in the word column
word_column_words = list(chain.from_iterable(data_frame['word'].str.lower().str.split()))
unigrams = set(word for word in word_column_words)
bigrams = set(nltk.ngrams(word_column_words, 2))
trigrams = set(nltk.ngrams(word_column_words, 3))
fourgrams = set(nltk.ngrams(word_column_words, 4))
fivegrams = set(nltk.ngrams(word_column_words, 5))

# Count the occurrences of unigrams, bigrams, trigrams, fourgrams, and fivegrams in the Simple Wiki corpus
file_path = "corpus/simple_wiki.txt"
unigram_occurrences = search_ngrams(file_path, unigrams)
bigram_occurrences = search_ngrams(file_path, bigrams)
trigram_occurrences = search_ngrams(file_path, trigrams)
fourgram_occurrences = search_ngrams(file_path, fourgrams)
fivegram_occurrences = search_ngrams(file_path, fivegrams)

# Combine the occurrence counts of all ngrams into a single column
data_frame['simple_wiki_freq'] = (
    data_frame['word'].apply(lambda x: sum(unigram_occurrences[word] for word in x.lower().split() if word in unigram_occurrences)) +
    data_frame['word'].apply(lambda x: sum(bigram_occurrences[bigram] for bigram in nltk.ngrams(x.lower().split(), 2) if bigram in bigram_occurrences)) +
    data_frame['word'].apply(lambda x: sum(trigram_occurrences[trigram] for trigram in nltk.ngrams(x.lower().split(), 3) if trigram in trigram_occurrences)) +
    data_frame['word'].apply(lambda x: sum(fourgram_occurrences[fourgram] for fourgram in nltk.ngrams(x.lower().split(), 4) if fourgram in fourgram_occurrences)) +
    data_frame['word'].apply(lambda x: sum(fivegram_occurrences[fivegram] for fivegram in nltk.ngrams(x.lower().split(), 5) if fivegram in fivegram_occurrences))
)


In [36]:
import pandas as pd
from collections import Counter
import nltk

# Function to count ngram occurrences in a file
def search_ngrams(file_path, ngrams):
    # Create a Counter object to store ngram occurrences
    ngram_occurrences = Counter()

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Convert to lowercase and split the line into words
            words = nltk.word_tokenize(line.lower())
            # Generate ngrams
            line_ngrams = list(nltk.ngrams(words, len(ngrams[0])))
            # Count ngram occurrences
            ngram_occurrences.update(line_ngrams)

    return ngram_occurrences


# Create a list of ngrams from the words in the word column
word_column_words = data_frame['word'].str.lower().str.split()
unigrams = [word for words in word_column_words for word in words]
bigrams = list(nltk.ngrams(word_column_words, 2))
trigrams = list(nltk.ngrams(word_column_words, 3))
fourgrams = list(nltk.ngrams(word_column_words, 4))
fivegrams = list(nltk.ngrams(word_column_words, 5))

# Count the occurrences of unigrams, bigrams, trigrams, fourgrams, and fivegrams in the Simple Wiki corpus
file_path = "corpus/simple_wiki.txt"
unigram_occurrences = search_ngrams(file_path, unigrams)
bigram_occurrences = search_ngrams(file_path, bigrams)
trigram_occurrences = search_ngrams(file_path, trigrams)
fourgram_occurrences = search_ngrams(file_path, fourgrams)
fivegram_occurrences = search_ngrams(file_path, fivegrams)

# Combine the occurrence counts of all ngrams into a single column
data_frame['simple_wiki_freq'] = (
    data_frame['word'].apply(lambda x: sum(unigram_occurrences[word] for word in x.lower().split())) +
    data_frame['word'].apply(lambda x: sum(bigram_occurrences[bigram] for bigram in nltk.ngrams(x.lower().split(), 2))) +
    data_frame['word'].apply(lambda x: sum(trigram_occurrences[trigram] for trigram in nltk.ngrams(x.lower().split(), 3))) +
    data_frame['word'].apply(lambda x: sum(fourgram_occurrences[fourgram] for fourgram in nltk.ngrams(x.lower().split(), 4))) +
    data_frame['word'].apply(lambda x: sum(fivegram_occurrences[fivegram] for fivegram in nltk.ngrams(x.lower().split(), 5)))
)


In [37]:
data_frame

,ID,sentence,start_index,end_index,word,total_native,total_non_native,native_complex,non_native_complex,complex_binary,complex_probabilistic,split,simple_wiki_freq
0,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",4,10,barren,10,10,6,2,1,0.4,[barren],0
1,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",4,18,barren islands,10,10,0,1,1,0.05,"[barren, islands]",0
2,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",20,25,reefs,10,10,1,2,1,0.15,[reefs],0
3,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",11,18,islands,10,10,0,0,0,0.0,[islands],0
4,3P7RGTLO6EE07HLUVDKKHS6O7CCKA5,"The barren islands, reefs and coral outcrops a...",30,35,coral,10,10,0,0,0,0.0,[coral],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13997,3BDORL6HKKEAN0VN5BP8OZIQ163CR1,"Stephen Biddle, a defense analyst at the Counc...",185,201,security partner,10,10,2,0,1,0.1,"[security, partner]",0
13998,3BDORL6HKKEAN0VN5BP8OZIQ163CR1,"Stephen Biddle, a defense analyst at the Counc...",185,193,security,10,10,0,0,0,0.0,[security],0
13999,3BDORL6HKKEAN0VN5BP8OZIQ163CR1,"Stephen Biddle, a defense analyst at the Counc...",194,201,partner,10,10,0,0,0,0.0,[partner],0
14000,3BDORL6HKKEAN0VN5BP8OZIQ163CR1,"Stephen Biddle, a defense analyst at the Counc...",217,223,troops,10,10,0,0,0,0.0,[troops],0


In [ ]:
data_frame.to_csv('output_D.csv', index=False)

In [ ]:
#code to work with pickled data

import pandas as pd
import string



data_frame = pd.read_pickle(location)
data_frame.columns = ['ID', 'sentence', 'start_index', 'end_index', 'word', 'total_native',
                      'total_non_native', 'native_complex', 'non_native_complex', 'complex_binary',
                      'complex_probabilistic']

# Cleaning function for words
remove = string.punctuation.replace("'", "")  # Remove all punctuation except apostrophes
remove = remove.replace("“", "").replace("”", "")  # Remove double quotation marks
pattern = r"[{}]".format(remove)  # Create the pattern

# Split the words and add them to the 'split' column, treating hyphens as separate words
data_frame['split'] = data_frame['word'].apply(lambda x: [word for word in x.replace('-', ' - ').split()])

print("Cleaned DataFrame:")
print(data_frame)


In [ ]:
# Gives number of vowels, syllables and characters for single word ONLY
#Removes the rows with MWEs but does return data for the individual words that the MWE is made up of
# 100,000 requests per day.
#We create a table that contains only the words
words = data_frame[data_frame['count'] == 1]

word_set = words.word.str.lower().unique()

word_set = pd.DataFrame(word_set)
word_set.columns = ['word']

api = datamuse.Datamuse()

print("Getting syllables, length and vowels")

def get_syllables(word):
    syllables = 0
    word_results = api.words(sp=word, max=1, md='psf')
    if len(word_results) > 0:
        word = word_results[0]["word"]
        syllables = int(word_results[0]["numSyllables"])
    return syllables

# Create a copy of the DataFrame with unique words
unique_words = word_set['word'].unique()
unique_word_set = pd.DataFrame(unique_words, columns=['word'])

# Apply function to get syllables for unique words
unique_word_set['syllables'] = unique_word_set['word'].apply(lambda x: get_syllables(x))

# Apply function to get word length
unique_word_set['length'] = unique_word_set['word'].apply(lambda x: len(x))

# Apply function to get vowel count
unique_word_set['vowels'] = unique_word_set['word'].apply(lambda x: sum([x.count(y) for y in "aeiou"]))

# Apply function to get consonant count
unique_word_set['consonants'] = unique_word_set['word'].apply(lambda x: sum([x.count(y) for y in "bcdfghjklmnpqrstvwxyz"]))

# Merge unique_word_set DataFrame with word_features DataFrame
words = words.copy()  # Create a copy of the DataFrame
words.loc[:, 'original word'] = words['word']
words.loc[:, 'word'] = words['word'].str.lower()
word_features = pd.merge(words, unique_word_set, on='word', how='left')

print(word_features)
print('Finished getting syllables, length and vowels', "\n")


In [ ]:
#Gives number of vowels, syllables and characters for all data

#second attempt at getting syllables for hyphenated words

def get_total_syllables(words):
    api = datamuse.Datamuse()
    total_syllables = 0
    word_list = words.split()
    for word in word_list:
        if '-' in word:
            hyphenated_words = word.split('-')
            for hyphenated_word in hyphenated_words:
                word_results = api.words(sp=hyphenated_word, max=1, md='psf')
                if len(word_results) > 0:
                    word = word_results[0]["word"]
                    syllables = int(word_results[0]["numSyllables"])
                    total_syllables += syllables
        else:
            word_results = api.words(sp=word, max=1, md='psf')
            if len(word_results) > 0:
                word = word_results[0]["word"]
                syllables = int(word_results[0]["numSyllables"])
                total_syllables += syllables
    return total_syllables

data_frame['syllables'] = data_frame['word'].apply(get_total_syllables)
data_frame['vowels'] = data_frame['word'].apply(lambda x: sum(1 for char in x if char.lower() in "aeiou"))
data_frame['characters'] = data_frame['word'].apply(len)

# Print values for each word
for index, row in data_frame.iterrows():
    print(f"Word: {row['word']}, Syllables: {row['syllables']}, Vowels: {row['vowels']}, Characters: {row['characters']}")

print("Updated DataFrame:")
print(data_frame)

In [ ]:
data_frame

In [ ]:
#get word freq using ngram download from https://www.kaggle.com/datasets/wheelercode/english-word-frequency-list/code?resource=download


import pandas as pd

def get_word_count(word, ngram_freq_file):
    df = pd.read_csv(ngram_freq_file)
    word_count = df.loc[df['word'] == word, 'count'].sum()
    return word_count


#ngram_freq_file = 'ngram_freq.csv'
# Assuming 'data_frame' is your existing DataFrame with a 'word' column
data_frame['ngram_freq'] = data_frame['word'].apply(lambda x: get_word_count(x, 'corpus/ngram_freq.csv') if len(x.split()) == 1 else None)



In [ ]:
def get_word_length_distribution():
    # Read the ngram_freq.csv file
    df = pd.read_csv('corpus/ngram_freq.csv')

    # Filter out float values in the 'word' column
    df = df[df['word'].apply(lambda x: isinstance(x, str))]

    # Calculate word lengths
    df['word_length'] = df['word'].apply(lambda x: len(x))

    # Group the data by word length and count the number of words
    word_length_counts = df['word_length'].value_counts().sort_index()

    # Calculate the total number of words
    total_words = word_length_counts.sum()

    # Create a DataFrame with the word length counts
    word_length_table = pd.DataFrame({'Word Length': word_length_counts.index, 'Count': word_length_counts.values})

    # Add a row for the total number of words
    total_row = pd.DataFrame({'Word Length': ['Total'], 'Count': [total_words]})
    word_length_table = word_length_table.append(total_row, ignore_index=True)

    return word_length_table

# Call the function to get the word length distribution table
word_length_table = get_word_length_distribution()

# Filter the table to keep only words with a length of 50
words_with_length_50 = word_length_table[word_length_table['Word Length'] == 40]

# Get the actual words with a length of 50
words = pd.read_csv('corpus/ngram_freq.csv')
words_with_length_50_words = words[words['word'].apply(lambda x: len(str(x)) == 40)]['word']

# Print the words with a length of 50
# Print the words with a length of 50
print("Words with a length of 20:")
print(words_with_length_50_words)

In [ ]:
# get bigram data https://github.com/orgtre/google-books-ngram-frequency/blob/main/ngrams/2grams_english.csv
#this is the most common bigrams data- 5.000 most frequent 2-grams








In [ ]:
#Experimenting with more accurate syllable count than Datamuse with NLTK- Conclusion was that Datamuse will be kept because that was what best-performing
#CAMB system used. THIS WAS FOUND TO BE LESS EFFECTIVE


# def get_syllables(word):
#     syllables = 0
#     word_results = api.words(sp=word, max=1, md='psf')
#     if len(word_results) > 0:
#         word = word_results[0]["word"]
#         syllables = int(word_results[0]["numSyllables"])
#     print("# of syllables for", word, ":", syllables)
#     return syllables



# nltk.download('cmudict')
# def get_syllablesNLTK(word):
#     cmudict = nltk.corpus.cmudict.dict()
#     syllables = 0
#     if word.lower() in cmudict:
#         syllables = [len(list(y for y in x if y[-1].isdigit())) for x in cmudict[word.lower()]]
#         syllables = max(syllables) if syllables else 0
#     return syllables
# get_syllablesNLTK('epidexipteryx')

In [ ]:
#Need to find a legit way to get this data

#Now for SimpleWiki Freqency
#https://www.loc.gov/item/2019205402/

#Need the wiki.simple dataset to run

# from nltk.corpus import stopwords
# #stop = stopwords.words('english')
# nltk.download('stopwords')
"""
Function to create Simple Wiki corpus 

"""
# def simple_wiki():

#     df_wiki = pd.read_table('wiki.simple', names=['sentence'])

#     # casing
#     df_wiki['sentence'] = df_wiki.sentence.str.lower()
#     # remove stop words
#     df_wiki['sentence'] = df_wiki.sentence.apply(lambda x: ' '.join(
#         [word for word in x.split() if word not in (stop)]))

#     # remove punctuaction
#     df_wiki['sentence'] = df_wiki.sentence.str.replace('[^\w\s]', '')
#     # print(df_wiki)

#     # remove numbers
#     df_wiki['sentence'] = df_wiki.sentence.str.replace('\d+', '')
#     df_wiki['sentence'] = df_wiki.sentence.str.strip()

#     series_top = pd.Series(
#         " ".join((df_wiki.sentence).str.lower()).split()).value_counts()

#     df_top = series_top.to_frame(name="frequency")
#     df_top['word'] = df_top.index
#     df_top['word'] = df_top.word[df_top.word.str.len() > 2]
#     df_top = df_top.dropna(axis=0)
#     print(df_top)

#     df_top = df_top.nlargest(6386, "frequency")
#     print(df_top)

#     df_top = df_top[['word', 'frequency']]

#     # # # make csv file
#     df_top.to_csv("simple_wiki.csv", index=False)


In [ ]:
# def count_word_occurrences(file_path, word):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         text = file.read()
#     count = text.lower().count(word.lower())
#     return count

# # Specify the file path of simple_wiki.txt
# file_path = "corpus/simple_wiki.txt"

# # Apply the function to count occurrences and add the column to word_features DataFrame
# word_features['simple_wiki_freq'] = word_features['word'].apply(lambda x: count_word_occurrences(file_path, x))


In [ ]:
#This only works for single words
"""
Counts all word occurrences in the simple Wiki corpus and stores the counts in a dictionary then looks up the count for each word in the dictionary. 
For a single word
"""
def count_all_word_occurrences(file_path):
    # Create a dictionary to store word counts
    word_counts = defaultdict(int)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Convert to lowercase and split the line into words
            words = re.findall(r'\b\w+\b', line.lower())
            # Count each word
            for word in words:
                word_counts[word] += 1
                
    return word_counts

# Count all word occurrences in the file
file_path = "corpus/simple_wiki.txt"
word_counts = count_all_word_occurrences(file_path)

# Look up the count for each word in the dictionary and add it to the DataFrame
data_frame['simple_wiki_freq'] = data_frame['word'].apply(lambda x: word_counts[x.lower()])


In [ ]:
data_frame

In [ ]:
#THIS CAUSED KERNAL TO QUIT
#This looks up bigrams trigams fourgrams and five grams in simple_wiki corpus

import itertools
from itertools import chain

def count_word_combinations(file_path, combinations):
    # Create a dictionary to store combination counts
    combination_counts = defaultdict(int)

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.lower()
            for combination in combinations:
                count = line.count(combination)
                if count > 0:
                    combination_counts[combination] += count

    return combination_counts




# Generate all combinations of words up to a maximum of five words
combinations = []

# Flatten the word_column_words list
word_column_words = list(chain.from_iterable(data_frame['word'].str.lower().str.split()))

# Generate all combinations of words up to a maximum of five words
combinations = []
for n in range(1, 6):
    combinations.extend([' '.join(combination) for combination in itertools.combinations(word_column_words, n)])
    
# Provide the path to the Simple Wiki corpus file
file_path = "corpus/simple_wiki.txt"

# Count the occurrences of the word combinations
combination_counts = count_word_combinations(file_path, combinations)

# Update the DataFrame with the combination counts
data_frame['combination_count'] = data_frame['word'].apply(lambda x: combination_counts.get(x.lower(), 0))






In [ ]:
data_frame.to_csv('output.csv', index=False)

In [ ]:
#The count_all_word_occurrences function is modified to count unigrams, bigrams, trigrams and hyphenated words in simple Wikipedia.
"""
In this updated version, the occurrence counts of unigrams, bigrams, trigrams, fourgrams, and fivegrams are calculated 
separately and then combined into a single column called simple_wiki_freq. The counts are obtained by summing the 
occurrence counts of each ngram type using the apply method with lambda functions.
"""

def search_ngrams(file_path, ngrams):
    # Create a dictionary to store ngram occurrences
    ngram_occurrences = defaultdict(int)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Convert to lowercase and split the line into words
            words = nltk.word_tokenize(line.lower())
            # Generate ngrams
            line_ngrams = list(nltk.ngrams(words, len(ngrams[0])))
            # Search for each ngram occurrence
            for ngram in line_ngrams:
                if ngram in ngrams:
                    ngram_occurrences[ngram] += 1
                
    return ngram_occurrences

# Create a list of ngrams from the words in the word column
word_column_words = data_frame['word'].str.lower().str.split()
unigrams = [word for words in word_column_words for word in words]
bigrams = list(nltk.ngrams(word_column_words, 2))
trigrams = list(nltk.ngrams(word_column_words, 3))
fourgrams = list(nltk.ngrams(word_column_words, 4))
fivegrams = list(nltk.ngrams(word_column_words, 5))

# Count the occurrences of unigrams, bigrams, trigrams, fourgrams, and fivegrams in the Simple Wiki corpus
file_path = "corpus/simple_wiki.txt"
unigram_occurrences = search_ngrams(file_path, unigrams)
bigram_occurrences = search_ngrams(file_path, bigrams)
trigram_occurrences = search_ngrams(file_path, trigrams)
fourgram_occurrences = search_ngrams(file_path, fourgrams)
fivegram_occurrences = search_ngrams(file_path, fivegrams)

# Combine the occurrence counts of all ngrams into a single column
data_frame['simple_wiki_freq'] = (
    data_frame['word'].apply(lambda x: sum(unigram_occurrences[word] for word in x.lower().split())) +
    data_frame['word'].apply(lambda x: sum(bigram_occurrences[bigram] for bigram in nltk.ngrams(x.lower().split(), 2))) +
    data_frame['word'].apply(lambda x: sum(trigram_occurrences[trigram] for trigram in nltk.ngrams(x.lower().split(), 3))) +
    data_frame['word'].apply(lambda x: sum(fourgram_occurrences[fourgram] for fourgram in nltk.ngrams(x.lower().split(), 4))) +
    data_frame['word'].apply(lambda x: sum(fivegram_occurrences[fivegram] for fivegram in nltk.ngrams(x.lower().split(), 5)))
)


In [ ]:
#data_frame.iloc[100:130]

In [ ]:
#WORKING Code
#Frequency in HIT paragraph

# def HIT_freq(data_frame, word, ID):
#     paragraph = data_frame[data_frame['ID'] == ID]
#     word_count = paragraph['word'].str.lower().value_counts().to_dict()
#     word_occurrences = word_count.get(word.lower(), 0)
#     return word_occurrences

# # Add word occurrences to word_features DataFrame
# word_features['HIT_count'] = word_features.apply(lambda row: HIT_freq(data_frame, row['word'], row['ID']), axis=1)



In [ ]:
#NEW CODE to get HIT frquency with MWEs

def HIT_freq(data_frame, words, ID):
    paragraph = data_frame[data_frame['ID'] == ID]
    word_count = paragraph['word'].str.lower().value_counts().to_dict()
    total_occurrences = 0
    for word in words.split():
        word_occurrences = word_count.get(word.lower(), 0)
        total_occurrences += word_occurrences
    return total_occurrences

# Add word occurrences to data_frame
data_frame['HIT_count'] = data_frame.apply(lambda row: HIT_freq(data_frame, row['word'], row['ID']), axis=1)


In [ ]:
data_frame.to_csv('output.csv', index=False)

In [ ]:
#Google Web IT 5-gram frequency was not freely available so this is got throght the Datamuse API and uses Google N-gram corpus
#Accessing Google's Web 5-gram corpus directly is not possible as the dataset is not publicly available.

#Need to parse sentence to return frquency using Datamuse api


# import requests

# def get_datamuse_frequency(word):
#     api_url = f"https://api.datamuse.com/words?sp={word}&md=f"
    
#     try:
#         response = requests.get(api_url)
#         data = response.json()
#         if isinstance(data, list) and len(data) > 0:
#             frequency = data[0].get('f', 0)
#             return frequency
#     except (requests.exceptions.RequestException, KeyError) as e:
#         print("Error occurred:", e)
    
#     return 0  # Default value when encountering an error

# # Apply the function to get Datamuse frequency for each word in the "word" column
# word_features['Datamuse frequency'] = word_features['word'].apply(lambda x: get_datamuse_frequency(x))

# word_features

In [ ]:
print("start core")
nlp = StanfordCoreNLP('http://localhost:9000')

sentences = data_frame[['sentence', 'ID']].copy()

sentences = sentences1.drop_duplicates()

print("end core")

In [ ]:
import pandas as pd

# Assuming you have a dataframe named 'data_frame' with a column named 'sentence'

# Function to parse a sentence and return the parsed output as a string
def parse_sentence(sentence):
    parsed_output = parse(sentence)
    return json.dumps(parsed_output)

# Apply the parse_sentence function to the 'sentence' column and store the parsed output in a new 'Parse' column
data_frame['Parse'] = data_frame['sentence'].apply(parse_sentence)

# Print the updated dataframe
data_frame


In [ ]:
sentences

In [ ]:
sentences

In [ ]:
print("start first token")

def removefirsttoken(x):
    x = x.split(' ', 1)[1]
    return x

sentences['clean sentence'] = sentences['sentence']

# sentences.to_csv("debugging/sentences_noparse.csv", index=False) debugging

print("start end token")

In [ ]:
sentences

In [ ]:
 # function to parse sentences
print("start parse sentence")

def parse(string):
    output = nlp.annotate(string, properties={
        'annotators': 'pos,depparse,ner',
        'outputFormat': 'json'
    })
    output_dict = json.loads(output)
    return output_dict
    # return output

# apply parsing to sentences
sentences['parse'] = sentences['clean sentence'].apply(lambda x: parse(x))
#MERGING THESE -  The issue is related to the parse column in the word_parse_features DataFrame, which should contain dictionaries representing the parsed output from the Stanford CoreNLP library.
word_parse_features = pd.merge(sentences, data_frame)
#word_parse_features.to_csv("debugging/word_parse_features_bug.csv" , index=False) 

print("finish parsing sentence")

In [ ]:
sentences

In [ ]:
data_frame

In [ ]:
def get_pos(row):
    word = row['word']
    parse = row['parse']
    pos = parse['sentences'][0]['tokens'][0]['pos']
    print(f"Token {i+1}: {pos}") 
    for i in range(len(parse['sentences'][0]['tokens'])):


        comp_word = parse['sentences'][0]['tokens'][i]['word']
        comp_word = comp_word.lower()
        comp_word = comp_word.translate(
            {ord(char): None for char in remove})
        print(parse['sentences'][0]['tokens'][i]['pos'])
        if comp_word == word:

            return parse['sentences'][0]['tokens'][i]['pos']
        


In [ ]:
def get_pos(row):
    word = row['word']
    parse = row['parse']
    
    for token in parse['sentences'][0]['tokens']:
        comp_word = token['word']
        comp_word = comp_word.lower()
        comp_word = comp_word.translate({ord(char): None for char in remove})
        
        if comp_word == word:
            return token['pos']
    
    return None


In [ ]:

['pos'] =data_frame.apply(get_pos, axis=1)

In [ ]:
def get_dep(row):
        number = 0
        word = row['word']
        parse = row['parse']
        for i in range(len(parse['sentences'][0]['basicDependencies'])):
            comp_word = parse['sentences'][0]['basicDependencies'][i]['governorGloss']
            comp_word = comp_word.lower()
            comp_word = comp_word.translate(
                {ord(char): None for char in remove})

            if comp_word == word:
                number += 1

        return number

In [ ]:
sentences['dep num'] = word_parse_features.apply(get_dep, axis=1)
word_parse_features

In [ ]:
def get_ner(row):
        word = row['word']
        parse = row['parse']
        # print(parse, "TEDFS")
        # print(parse['sentences'][0]['tokens'], "DOFISPFDSIU")
        # print(parse['sentences'][1]['tokens'], "DOFISPFDSIU")

        for i in range(len(parse['sentences'][0]['tokens'])):
            comp_word = parse['sentences'][0]['tokens'][i]['word']
            comp_word = comp_word.lower()
            comp_word = comp_word.translate(
                {ord(char): None for char in remove})
            print(parse['sentences'][0]['tokens'][i]['ner'])
            if comp_word == word:
                
                return parse['sentences'][0]['tokens'][i]['ner']
                

In [ ]:
sentences['ner'] = word_parse_features.apply(get_ner, axis=1)

In [ ]:
word_parse_features

In [ ]:
print("start get pos")
word_parse_features['pos'] = word_parse_features.apply(get_pos, axis=1)
print("end get pos")

print("start get dep")

word_parse_features['dep num'] = word_parse_features.apply(get_dep, axis=1)
word_parse_features['ner'] = word_parse_features.apply(get_ner, axis=1)

print("end get dep")

In [ ]:
word_parse_features

In [ ]:
 # Function to get the proper lemma

print("start tagging")
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    if pd.isna(treebank_tag):
        return None

    if isinstance(treebank_tag, str) and treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif isinstance(treebank_tag, str) and treebank_tag.startswith('V'):
        return wordnet.VERB
    elif isinstance(treebank_tag, str) and treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif isinstance(treebank_tag, str) and treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


In [ ]:
print("start lemmatizing")
    
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatiser(row):
    word = row['word']
    pos = row['pos']
    lemmatizer = WordNetLemmatizer()
    wordnet_pos = get_wordnet_pos(pos)

    if wordnet_pos:
        lemma = lemmatizer.lemmatize(word, wordnet_pos)
        return lemma
    else:
        lemma = lemmatizer.lemmatize(word)
        return lemma
print("finish lemmatizing", "\n")

In [ ]:
  # Convert tree bank tags to ones that are compatible w google


def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']

def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']

def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']

def penn_to_wn(tag):
    if is_adjective(tag):
        return wordnet.ADJ
    elif is_noun(tag):
        return wordnet.NOUN
    elif is_adverb(tag):
        return wordnet.ADV
    elif is_verb(tag):
        return wordnet.VERB
    return None

def penn_to_google(tag):
    if is_adjective(tag):
        return 'adj'
    elif is_noun(tag):
        return 'n'
    elif is_adverb(tag):
        return 'adv'
    elif is_verb(tag):
        return 'v'
    return None

In [ ]:
def get_frequency(row):
        nofreq = float(0.000000)
        word = row["word"]
        print("word:", word)
        word = str(word)
        tag = row["pos"]
        tag = penn_to_google(tag)

        try:
            word_results = api.words(sp=word, max=1, md='pf')
            tag_list = (word_results[0]['tags'][:-1])

            frequency = word_results[0]['tags'][-1][2:]

            frequency = float(frequency)

            if tag in tag_list:
                print("frequency_1:", frequency)
                return frequency
            else:
                lemma = row['lemma']
                try:
                    word_results = api.words(sp=lemma, max=1, md='pf')
                    tag_list = (word_results[0]['tags'][:-1])
                    frequency = word_results[0]['tags'][-1][2:]
                    frequency = float(frequency)

                    if tag in tag_list:
                        print("frequency_2:", frequency)
                        return frequency
                    else:
                        print("nofreq")
                        return nofreq
                except:
                    return nofreq

        except:

            return nofreq


In [ ]:
word_parse_features

In [ ]:
# Iterate over the "word" column in the DataFrame and calculate the frequency
data_frame['frequency'] = word_parse_features.apply(get_frequency, axis=1)

In [ ]:
# Apply function for google freq
print("get google frequency")
data_frame['google frequency'] = word_parse_features.apply(
    get_frequency, axis=1)
print("end google frequency")

In [ ]:
data_frame

In [ ]:
import os
import pickle

# Get the base filename from the location variable
base_filename = os.path.splitext(os.path.basename(location))[0]

# Specify the output file path
output_file_path = os.path.join('features_NEW', base_filename + '_NEW_Feats.pkl')

# Pickle the word_features DataFrame
with open(output_file_path, 'wb') as file:
    pickle.dump(data_frame, file)

In [ ]:
import pandas as pd

def print_rows_with_zero(data_frame):
    for index, row in data_frame.iterrows():
        ngram_freq = row['ngram_freq']
        if ngram_freq == 0:
            print(row)

In [ ]:
print_rows_with_zero(data_frame)

In [ ]:
def count_zeros_in_column(data_frame, column_name):
    count = 0
    for index, row in data_frame.iterrows():
        value = row[column_name]
        if value == 0:
            count += 1
    return count

In [ ]:
count_zeros_in_column(data_frame, "ngram_freq")

In [16]:
all_levels = pd.read_table('corpus/cefrj-vocabulary-profile-1.5.csv', names=('headword', 'CEFR'))

def levels(word):
    word = ''.join(word.split()).lower()
    try:
        df = all_levels.loc[all_levels['headword'] == word]
        level = df.iloc[0]['CEFR']
        return level

    except:
        try:
            df = all_levels.loc[all_levels['headword'] == word]
            level = df.iloc[0]['CEFR']
            return level
        except:
            return 0

In [20]:
levels("abandon")

'B1'

In [19]:
import pandas as pd

def levels(word):
    all_levels = pd.read_csv('corpus/cefrj-vocabulary-profile-1.5.csv')
    word = ''.join(word.split()).lower()
    df = all_levels.loc[all_levels['headword'] == word]
    if not df.empty:
        level = df.iloc[0]['CEFR']
        return level
    else:
        return 0

In [ ]:
levels(